In [ ]:
!pip install bitsandbytes
!pip install -q datasets loralib sentencepiece accelerate
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip install -q git+https://github.com/huggingface/peft.git

In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, LLaMAForCausalLM, LLaMATokenizer
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model

In [ ]:
MICRO_BATCH_SIZE = 4
BATCH_SIZE = 128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 2  # we don't need 3 tbh
LEARNING_RATE = 3e-4  # the Karpathy constant
CUTOFF_LEN = 256  # 256 accounts for about 96% of the data
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

# Please specify a BASE_MODEL, e.g. BASE_MODEL = 'decapoda-research/llama-7b-hf'
BASE_MODEL = ""
# Insert the path to your data
DATA_PATH = "mchl-labs/stambecco_data_plus_it"
# Insert the name of your model
NAME = "stambecco-7b-plus"
# Insert your HUGGINGFACE HUB Token
os.environ["HUB_TOKEN"] = "YOUR_HUB_TOKEN_HERE"

In [ ]:
hub_token = os.environ["HUB_TOKEN"]
print(f"Hub token: {hub_token}")

In [ ]:
model = LLaMAForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    device_map="auto",
)

tokenizer = LLaMATokenizer.from_pretrained(
    BASE_MODEL, add_eos_token=True
)

model = prepare_model_for_int8_training(model)

config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
tokenizer.pad_token_id = 0  # We want this to be different from the eos token

In [ ]:
if DATA_PATH.endswith(".json") or DATA_PATH.endswith(".jsonl"):
        data = load_dataset("json", data_files=DATA_PATH)
else:
        data = load_dataset(DATA_PATH)

In [ ]:
def generate_prompt(data_point):
    if data_point["input"]:
        return f"""Di seguito è riportata un'istruzione che descrive un task, insieme ad un input che fornisce un contesto più ampio. Scrivi una risposta che completi adeguatamente la richiesta.
### Istruzione:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Risposta:
{data_point["output"]}"""
    else:
        return f"""Di seguito è riportata un'istruzione che descrive un task. Scrivi una risposta che completi adeguatamente la richiesta.
### Istruzione:
{data_point["instruction"]}
### Risposta:
{data_point["output"]}"""



def tokenize(prompt):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length",
    )
    return {
        "input_ids": result["input_ids"][:-1],
        "attention_mask": result["attention_mask"][:-1],
    }

In [ ]:
data = data.shuffle().map(lambda x: tokenize(generate_prompt(x)))

In [ ]:
model.print_trainable_parameters()

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=100,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=20,
        optim="adamw_torch",
        output_dir=NAME,
        save_total_limit=3,
        save_strategy="epoch",
        # push_to_hub=True,
        # hub_token=hub_token,
        # hub_private_repo=True,
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train(resume_from_checkpoint=False)

In [ ]:
model.save_pretrained(f"models/{NAME}")
# trainer.push_to_hub(language="it", license="cc-by-nc-nd-4.0", finetuned_from="llama-7b", dataset=DATA_PATH)

In [ ]:
model.push_to_hub(repo_id=NAME, use_auth_token=hub_token)